In [26]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [27]:
def scraping_utmb():
    url = 'https://utmb.world/utmb-index/runner-search'
    browsers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 \(KHTML, like Gecko) Chrome / 86.0.4240.198Safari / 537.36'}
    page = requests.get(url, headers=browsers)

    soup = BeautifulSoup(page.content, 'html.parser')
    runners = soup.select('.my-table_row__nlm_j')

    dict_names = {
        i.find('a', attrs={'class': 'link_link__96ppl link-underline'}).text: [i.find('div', attrs={'class': 'my-table_cell__z__zN col-2'}).text,
        str(i.find('div', attrs={'data-label': "nationality"}))[180:182],
        i.find('div', attrs={'data-label': "Gender"}).text,
        i.find('div', attrs={'data-label': "Age Group"}).text, 
        'https://utmb.world/'+ str(i.find('a', attrs={'class': 'link_link__96ppl link-underline'})['href'])
        ]
        for i in runners
    }

    return dict_names

resultado = scraping_utmb()

df = pd.DataFrame(resultado.values(), index=resultado.keys())
df = df.set_axis(['Posição', 'Nacionalidade', 'Gênero', 'Faixa Etária', 'Site'], axis=1)
df


,Posição,Nacionalidade,Gênero,Faixa Etária,Site
Remi BONNET,943,ch,Men,20-34,https://utmb.world//en/runner/687094.remi.bonnet
Kilian JORNET BURGADA,940,es,Men,35-39,https://utmb.world//en/runner/2704.kilian.jorn...
Patrick KIPNGENO,939,ke,Men,20-34,https://utmb.world//en/runner/4595246.patrick....
Jonathan ALBON,938,gb,Men,35-39,https://utmb.world//en/runner/527631.jonathan....
Jim WALMSLEY,936,us,Men,20-34,https://utmb.world//en/runner/485740.jim.walmsley
Stian ANGERMUND,932,no,Men,35-39,https://utmb.world//en/runner/493528.stian.ang...
Adam PETERMAN,932,us,Men,20-34,https://utmb.world//en/runner/3572577.adam.pet...
Philemon KIRIAGO,932,ke,Men,20-34,https://utmb.world//en/runner/4595248.philemon...
Petter ENGDAHL,927,se,Men,20-34,https://utmb.world//en/runner/1065223.petter.e...
Elhousine ELAZZAOUI,925,ma,Men,20-34,https://utmb.world//en/runner/2161523.elhousin...


---
Selenium
---

In [28]:

from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager 
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from time import sleep

# Funções

In [29]:
def extrair_dados_pagina(driver):
    runners = driver.find_elements(by=By.CSS_SELECTOR, value=".my-table_row__nlm_j")
    
    dict_names = {}
    for runner in runners:
        name = runner.find_element(by=By.XPATH, value="//div[@class='index-ranking-table_cell_general__xqlqF my-table_cell__z__zN col-4']").text
        nationality = runner.find_element(by=By.XPATH, value="//div[@class='index-ranking-table_cell_nationality__Pskf9 my-table_cell__z__zN col-2 my-table_hide_mobile__t6aJD']").text
        gender = runner.find_element(by=By.XPATH, value="//a[@class='index-ranking-table_cell_gender__svRts my-table_cell__z__zN col-2 my-table_hide_mobile__t6aJD']").text
        age_group = runner.find_element(by=By.XPATH, value="//a[@class='index-ranking-table_cell_age_group__WOVLq my-table_cell__z__zN col-2 my-table_hide_mobile__t6aJD']").text
        link = runner.find_element(by=By.XPATH, value="//a[@href='.link_link__96ppl.link-underline]").text
        
        dict_names[name] = [nationality, gender, age_group, link]

    return dict_names


In [31]:
service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service)

driver.get(url='https://utmb.world/utmb-index/runner-search')

reject = driver.find_element(by=By.ID, value="onetrust-reject-all-handler")
reject.click()

parameters = driver.find_elements(by=By.XPATH, value="//div[@class='index-ranking-table_cell_nationality__Pskf9 my-table_cell__z__zN col-2 my-table_hide_mobile__t6aJD']")
parameter = parameters[10]
nacionalidade_button = driver.find_element(by=By.XPATH, value="//button[@aria-controls='menu--7']")
driver.execute_script("arguments[0].scrollIntoView();", parameter)
wait = WebDriverWait(driver, 10)
wait.until(EC.visibility_of_element_located((By.XPATH, "//div[@class='index-ranking-table_cell_nationality__Pskf9 my-table_cell__z__zN col-2 my-table_hide_mobile__t6aJD']")))
nacionalidade_button.click() 

brazil_option = driver.find_element(by=By.XPATH, value="//div[@id='option-27--menu--7']")
brazil_option.click()

current_page_data = extrair_dados_pagina(driver)


ElementClickInterceptedException: Message: element click intercepted: Element is not clickable at point (596, 971)
  (Session info: chrome=121.0.6167.160)
Stacktrace:
#0 0x561847348d93 <unknown>
#1 0x56184702c337 <unknown>
#2 0x56184707ee2e <unknown>
#3 0x56184707cbd0 <unknown>
#4 0x56184707a44a <unknown>
#5 0x561847079226 <unknown>
#6 0x56184706c6b7 <unknown>
#7 0x56184709a442 <unknown>
#8 0x56184706c028 <unknown>
#9 0x56184709a60e <unknown>
#10 0x5618470b912d <unknown>
#11 0x56184709a1e3 <unknown>
#12 0x56184706a135 <unknown>
#13 0x56184706b13e <unknown>
#14 0x56184730ce4b <unknown>
#15 0x561847310dfa <unknown>
#16 0x5618472f96d5 <unknown>
#17 0x561847311a6f <unknown>
#18 0x5618472dd69f <unknown>
#19 0x561847336098 <unknown>
#20 0x561847336262 <unknown>
#21 0x561847347f34 <unknown>
#22 0x7f875fd60ac3 <unknown>
